# Instalar librerias

In [2]:
!pip install aquacrop

In [34]:
!pip install catboost

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.6 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aquacrop 3.0.9 requires numpy==1.22.4, but you have numpy 1.26.4 which is incompatible.


# Cargar librerias

In [35]:
# Importar librerias
import calendar
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

from google.colab import drive
# Montar Google Drive como unidad local
drive.mount('/content/gdrive')

#from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, FieldMngt, GroundWater, IrrigationManagement
#from aquacrop.utils import prepare_weather, get_filepath
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from random import choice, randint
from tqdm import tqdm

from catboost import CatBoostRegressor

sns.set_style("darkgrid")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Definición de funciones

In [36]:
# Filtrado de datos secuenciales disponibles.
# La función encuentra el período secuencial más largo disponible.

def find_longest_sequential_period(df, date_column):
    # Ordenar el dataFrame por la columna de fecha
    df = df.sort_values(by=date_column).reset_index(drop=True)

    max_period_start = None
    max_period_end = None
    max_period_length = 0
    current_period_start = None
    current_period_length = 0

    for index, row in df.iterrows():
        if current_period_start is None:
            current_period_start = row[date_column]
            current_period_length = 1
        else:
            prev_date = df.at[index - 1, date_column]
            if (row[date_column] - prev_date).days == 1:
                current_period_length += 1
            else:
                if current_period_length > max_period_length:
                    max_period_length = current_period_length
                    max_period_start = current_period_start
                    max_period_end = prev_date
                current_period_start = row[date_column]
                current_period_length = 1

    # Comprueba si el último periodo es el más largo
    if current_period_length > max_period_length:
        max_period_length = current_period_length
        max_period_start = current_period_start
        max_period_end = df.at[len(df) - 1, date_column]

    return max_period_start.strftime('%Y/%m/%d'), max_period_end.strftime('%Y/%m/%d')

In [37]:
# Los datos secuenciales disponibles no tienen información para los 365 días que el cultivo de caña de azúcar requiere.
# La función expande el dataframe a 365 días.

def expand_to_365(df, date_column):
    num_rows = df.shape[0]

    # If there are already 365 or more rows, return the first 365 rows
    if num_rows >= 365:
        return df.iloc[:365]

    # Calculate the number of rows to replicate
    rows_to_add = 365 - num_rows
    last_rows = df.iloc[-rows_to_add:]  # Take the last 'rows_to_add' rows

    # Generate additional rows by replicating the last 'rows_to_add' rows
    additional_rows = []
    for i in range(rows_to_add):
        new_row = last_rows.iloc[i % len(last_rows)].copy()
        new_date = last_rows.iloc[-1][date_column] + timedelta(days=(i + 1))
        new_row[date_column] = new_date
        additional_rows.append(new_row)

    # Create a DataFrame for the additional rows
    additional_df = pd.DataFrame(additional_rows, columns=df.columns)

    # Concatenate the original DataFrame with the additional rows
    expanded_df = pd.concat([df, additional_df], ignore_index=True)

    return expanded_df

In [38]:
# Función para reemplazar valores atípicos con los percentiles correspondientes

def replace_outliers_with_percentiles(x, percentile_ranges, columns=None):
    """
    Reemplazar los valores atípicos en un DataFrame o una serie de pandas con los valores de percentil especificados.

    Parámetros:
    - x: DataFrame o serie de pandas
    - percentile_ranges: diccionario con nombres de columnas como claves y tuplas (percentil_inferior, percentil_superior) como valores
    - columnas: lista de nombres de columnas a las que aplicar la operación (solo para DataFrame). Si es None, se utilizarán todas las columnas.

    Devuelve:
    - El DataFrame o la serie modificados con los valores atípicos reemplazados por valores de percentil
    """

    if isinstance(x, pd.DataFrame):
        if columns is None:
            columns = x.columns  # Utilizar todas las columnas si no se especifica

        for column in columns:
            if column in x.columns:
                if column in percentile_ranges:
                    lower_percentile, upper_percentile = percentile_ranges[column]
                    # Calcular los límites de percentil inferior y superior para la columna
                    lower_cap = x[column].quantile(lower_percentile / 100.0)
                    upper_cap = x[column].quantile(upper_percentile / 100.0)

                    # Reemplazar valores atípicos con límites de percentiles
                    x[column] = x[column].clip(lower=lower_cap, upper=upper_cap)
                else:
                    print(f"No se proporciona el rango de percentil para la columna '{column}'.")
            else:
                print(f"La columna '{column}' no existe en el DataFrame.")

    elif isinstance(x, pd.Series):
        if len(percentile_ranges) != 1:
            raise ValueError("Para una serie, el rango de percentil debe contener exactamente un elemento.")

        column, (lower_percentile, upper_percentile) = list(percentile_ranges.items())[0]
        # Calcular los límites de percentiles inferior y superior para la serie
        lower_cap = x.quantile(lower_percentile / 100.0)
        upper_cap = x.quantile(upper_percentile / 100.0)

        # Reemplazar valores atípicos con límites de percentiles
        x = x.clip(lower=lower_cap, upper=upper_cap)

    else:
        raise TypeError("La entrada debe ser un DataFrame o una serie de pandas.")

    return x

# Definición de parámetros

In [39]:
# Definir rutas (paths)
root_path = '/content/gdrive/MyDrive/Wito_20240428/master_thesis/'

data_path = root_path + 'data/'
models_path = root_path + 'models/'
output_path = root_path + 'output/'

In [40]:
# Nombre de la variable objetivo
target_name = 'fresh_yield'

# Nombre de las covariables
features_names = ['seasonal_irrigation', 'type_soil', 'MinTemp_min', 'MaxTemp_min', 'Precipitation_min', 'ReferenceET_min',
       'MinHmd_min', 'MaxHmd_min', 'Radsolar_min', 'MinTemp_max',
       'MaxTemp_max', 'Precipitation_max', 'ReferenceET_max', 'MinHmd_max',
       'MaxHmd_max', 'Radsolar_max', 'MinTemp_mean', 'MaxTemp_mean',
       'Precipitation_mean', 'ReferenceET_mean', 'MinHmd_mean', 'MaxHmd_mean',
       'Radsolar_mean', 'MinTemp_last_quarter_min', 'MaxTemp_last_quarter_min',
       'Precipitation_last_quarter_min', 'ReferenceET_last_quarter_min',
       'MinHmd_last_quarter_min', 'MaxHmd_last_quarter_min',
       'Radsolar_last_quarter_min', 'MinTemp_last_quarter_max',
       'MaxTemp_last_quarter_max', 'Precipitation_last_quarter_max',
       'ReferenceET_last_quarter_max', 'MinHmd_last_quarter_max',
       'MaxHmd_last_quarter_max', 'Radsolar_last_quarter_max',
       'MinTemp_last_quarter_mean', 'MaxTemp_last_quarter_mean',
       'Precipitation_last_quarter_mean', 'ReferenceET_last_quarter_mean',
       'MinHmd_last_quarter_mean', 'MaxHmd_last_quarter_mean',
       'Radsolar_last_quarter_mean']

In [41]:
# Descripción variable objetivo
target_name_val = 'Rendimiento_estimado'

# Lista descripción covariables
features_names_vals =\
['Riego_estacional', 'Tipo_suelo',

'TempMin_min', 'TempMax_min',
'Precipitacion_min', 'Evapotranspiracion_min',
'MinHumedRel_min', 'MaxHumedRel_min', 'RadSolar_min',

'TempMin_max', 'TempMax_max',
'Precipitacion_max', 'Evapotranspiracion_max',
'MinHumedRel_max', 'MaxHumedRel_max', 'RadSolar_max',

'TempMin_promedio', 'TempMax_promedio',
'Precipitacion_promedio', 'Evapotranspiracion_promedio',
'MinHumedRel_promedio', 'MaxHumedRel_promedio', 'RadSolar_promedio',

'TempMin_ult_trim_min', 'TempMax_ult_trim_min',
'Precipitacion_ult_trim_min', 'Evapotranspiracion_ult_trim_min',
'MinHumedRel_ult_trim_min', 'MaxHumedRel_ult_trim_min', 'RadSolar_ult_trim_min',

'TempMin_ult_trim_max', 'TempMax_ult_trim_max',
'Precipitacion_ult_trim_max', 'Evapotranspiracion_ult_trim_max',
'MinHumedRel_ult_trim_max', 'MaxHumedRel_ult_trim_max', 'RadSolar_ult_trim_max',

'TempMin_ult_trim_promedio', 'TempMax_ult_trim_promedio',
'Precipitacion_ult_trim_promedio', 'Evapotranspiracion_ult_trim_promedio',
'MinHumedRel_ult_trim_promedio', 'MaxHumedRel_ult_trim_promedio', 'RadSolar_ult_trim_promedio']

In [42]:
# Crear diccionario para renombrar la variable objetivo
dict_target = dict(zip([target_name], [target_name_val]))

# Crear diccionario para renombrar las variables
dict_features = dict(zip(features_names, features_names_vals))

In [43]:
# Crear diccionario para renombrar a español los nombres de los suelos ya que al inicio para la simulación,
# aquacrop los leee únicamente en inglés
dict_soil = {
             'ClayLoam': 'Franco Arcilloso',
             'Clay': 'Arcilloso',
             'SandyLoam': 'Franco Arenoso',
             'SiltClay': 'Arcillo Limoso',
             'SiltClayLoam': 'Franco Arcillo Limoso'
              }

# Preprocesamiento de datos

Info 2021

In [44]:
# Leer data
df_1 = pd.read_excel(data_path + "2021.xlsx")

# Seleccionar columnas necesarias
df_1 = df_1[['est', 'fc_obsrvcion', 'tmp_mnma', 'tmp_mxma', 'prcptcion', 'evp_clclada',
             'hmd_rel_mnma', 'hmd_rel_mxma', 'rdcion_slar']]

# Renombrar columnas
df_1 = df_1.rename(columns={'evp_clclada': 'etp'})

# Asegurarse de que la columna 'Date' esté en formato datetime
df_1['fc_obsrvcion'] = pd.to_datetime(df_1['fc_obsrvcion'])

# Crear nuevas columnas en el DataFrame para el año, mes y día
df_1['Year'] = df_1['fc_obsrvcion'].dt.year
df_1['Month'] = df_1['fc_obsrvcion'].dt.month
df_1['Day'] = df_1['fc_obsrvcion'].dt.day

print(df_1.shape)
print(df_1.columns)

(11548, 12)
Index(['est', 'fc_obsrvcion', 'tmp_mnma', 'tmp_mxma', 'prcptcion', 'etp',
       'hmd_rel_mnma', 'hmd_rel_mxma', 'rdcion_slar', 'Year', 'Month', 'Day'],
      dtype='object')


Info 2022

In [45]:
# Leer data
df_2 = pd.read_csv(data_path + "2022.csv",sep=',')

# Seleccionar columnas necesarias
df_2 = df_2[['est', 'fc_obsrvcion', 'tmp_mnma', 'tmp_mxma', 'prcptcion', 'evp_clclada',
             'hmd_rel_mnma', 'hmd_rel_mxma', 'rdcion_slar']]

# Renombrar columnas
df_2 = df_2.rename(columns={'evp_clclada': 'etp'})

# Asegurarse de que la columna 'Date' esté en formato datetime
df_2['fc_obsrvcion'] = pd.to_datetime(df_2['fc_obsrvcion'])

# Crear nuevas columnas en el DataFrame para el año, mes y día
df_2['Year'] = df_2['fc_obsrvcion'].dt.year
df_2['Month'] = df_2['fc_obsrvcion'].dt.month
df_2['Day'] = df_2['fc_obsrvcion'].dt.day

print(df_2.shape)
print(df_2.columns)

(8308, 12)
Index(['est', 'fc_obsrvcion', 'tmp_mnma', 'tmp_mxma', 'prcptcion', 'etp',
       'hmd_rel_mnma', 'hmd_rel_mxma', 'rdcion_slar', 'Year', 'Month', 'Day'],
      dtype='object')


Info Total

In [46]:
# Concatenar información en un único dataframe para análisis
df = pd.concat([df_2, df_1])
df = df.sort_values(by=['est', 'fc_obsrvcion']).reset_index(drop=True)

print(df.shape)
print(df.columns)

(19856, 12)
Index(['est', 'fc_obsrvcion', 'tmp_mnma', 'tmp_mxma', 'prcptcion', 'etp',
       'hmd_rel_mnma', 'hmd_rel_mxma', 'rdcion_slar', 'Year', 'Month', 'Day'],
      dtype='object')


# Cargar datos raster

In [47]:
# Leer data del mapa
df_raster = pd.read_csv(data_path + "Raster.txt", encoding="utf-16", sep="\t")
df_raster.shape

(42135, 6)

In [48]:
# Leer data de clima
df_clima = pd.read_csv(data_path + "2022.csv",sep=',')

In [49]:
dict_est = df_clima.groupby('name')['est'].apply(lambda x: next(iter(set(x)))).to_dict()

In [50]:
df_raster['Estacion'].unique()

array(['MELENDEZ', 'PTAR CALI', 'CANDELARIA', 'EL TIPLE', 'ORTIGAL',
       'BOCAS DE PALO', 'CENICANA', 'PRADERA', 'FLORIDA', 'MIRANDA',
       'AEROPUERTO', 'PALMIRA SAN JOSE'], dtype=object)

In [51]:
df_raster['Estacion'].map(dict_est).unique()

array(['MEL', 'PTA', 'CAN', 'TIP', 'ORT', 'BDP', 'CEN', 'PRA', 'FLO',
       'MIR', 'AER', 'PSJ'], dtype=object)

In [52]:
sum(df_raster['Estacion'].isna())

0

In [53]:
df_raster['Hoja2__TEXTURA'].unique()

array(['Franco Arcillosa', 'Arcillosa', 'Franco Arenosa',
       'Arcillo Limosa', 'Franco Arcillo Limosa'], dtype=object)

In [54]:
dict_suelos = {
                'Franco Arcillosa': 'ClayLoam',
                'Arcillosa': 'Clay',
                'Franco Arenosa': 'SandyLoam',
                'Arcillo Limosa': 'SiltClay',
                'Franco Arcillo Limosa': 'SiltClayLoam',
              }

In [55]:
df_raster['Hoja2__TEXTURA'].map(dict_suelos).unique()

array(['ClayLoam', 'Clay', 'SandyLoam', 'SiltClay', 'SiltClayLoam'],
      dtype=object)

In [56]:
sum(df_raster['Hoja2__TEXTURA'].isna())

0

# Cargar modelo

In [57]:
# Cargar modelo
infile = open(models_path + "CatBoost_model.pkl", 'rb')
cat_model_f = pickle.load(infile)

# Simulación Aquacrop

In [58]:
df_unique = df_raster[['Estacion', 'Hoja2__TEXTURA']].drop_duplicates().reset_index(drop=True)

In [ ]:
# Dataframe para guardar la simulación
df_prod_final = pd.DataFrame()

# Iterate through each row in the prediction DataFrame
for index, row in tqdm(df_unique.iterrows(), total=len(df_unique)):

    # Filtrar estaciones
    filter_est = dict_est[row['Estacion']]
    df_filtrado = df[df['est'] == filter_est].copy()

    # Asegurarse de que la columna 'Date' esté en formato datetime
    df_filtrado['fc_obsrvcion'] = pd.to_datetime(df_filtrado['fc_obsrvcion'])

    # Crear nuevas columnas en el DataFrame para el año, mes y día
    df_filtrado['Year'] = df_filtrado['fc_obsrvcion'].dt.year
    df_filtrado['Month'] = df_filtrado['fc_obsrvcion'].dt.month
    df_filtrado['Day'] = df_filtrado['fc_obsrvcion'].dt.day

    # Ordenar por fecha
    df_filtrado = df_filtrado.sort_values(by='fc_obsrvcion')

    # Seleccionar solo las columnas deseadas
    columnas_deseadas = ['Day', 'Month', 'Year', 'tmp_mnma', 'tmp_mxma', 'prcptcion', 'etp',
                         'hmd_rel_mnma', 'hmd_rel_mxma', 'rdcion_slar']

    # Crear un nuevo DataFrame con solo las columnas seleccionadas
    df_filtrado = df_filtrado[columnas_deseadas]

    # Cambiar el nombre de las columnas utilizando un diccionario
    df_filtrado = df_filtrado.rename(columns={'tmp_mnma': 'MinTemp',
                                              'tmp_mxma': 'MaxTemp',
                                              'prcptcion': 'Precipitation',
                                              'etp': 'ReferenceET',
                                              'hmd_rel_mnma': 'MinHmd',
                                              'hmd_rel_mxma': 'MaxHmd',
                                              'rdcion_slar': 'Radsolar',
                                              'fc_obsrvcion': 'Date'})

    # Restablecer índice
    df_filtrado.reset_index(drop=True, inplace=True)

    ################ Preparar el clima ################

    # Poner las fechas del clima en formato de fecha y hora
    df_filtrado["Date"] = pd.to_datetime(df_filtrado[["Year", "Month", "Day"]])

    # Eliminar las columnas de día, mes y año
    df_filtrado = df_filtrado.drop(["Day", "Month", "Year"], axis=1)

    # Restablecer índice
    df_filtrado.reset_index(drop=True, inplace=True)

    # Establezca un límite en ET0 para evitar errores de división por cero
    df_filtrado.ReferenceET.clip(lower=0.1, inplace=True)

    # Eliminar duplicados por fecha si los hay
    df_filtrado.drop_duplicates(['Date'], keep='last', inplace=True)

    # Encontrar el período secuencial más largo disponible
    result = find_longest_sequential_period(df_filtrado, 'Date')

    # Fecha de inicio y fin de simulación
    start_sim = result[0]
    end_sim = result[1]

    # Dataframe filtrado
    df_sim_filt = df_filtrado[(df_filtrado['Date'] >= start_sim) &
                              (df_filtrado['Date'] <= end_sim)]

    # Expandir el dataframe a 365 días
    df_sim = expand_to_365(df_sim_filt, 'Date')

    ################ Suelo ################

    # Filtrar suelos
    type_soil = dict_suelos[row['Hoja2__TEXTURA']]
    sandy_loam = Soil(soil_type=type_soil)

    ################ Cultivo ################

    plant_date = df_sim['Date'].min().strftime('%m/%d')
    harv_date = df_sim['Date'].max().strftime('%m/%d')

    sugarcane = Crop('SugarCane', planting_date=plant_date, harvest_date=harv_date)

    ################ Contenido inicial de agua ################

    InitWC = InitialWaterContent(value=['FC'])

    ################ Modelo ################

    # Modelo de aquacrop y especificación de fecha de inicio y finalización de la simulación
    model = AquaCropModel(sim_start_time=df_sim['Date'].min().strftime('%Y/%m/%d'),
                          sim_end_time=df_sim['Date'].max().strftime('%Y/%m/%d'),
                          weather_df=df_sim,
                          soil=sandy_loam,
                          crop=sugarcane,
                          initial_water_content=InitWC,
                          irrigation_management=IrrigationManagement(irrigation_method=1),
                          groundwater=GroundWater(method='Constant', values=[2]),
                          field_management=FieldMngt(mulches=True, mulch_pct=100))

    # Ejecutar el modelo hasta su terminación
    model.run_model(till_termination=True)

    # Calcular el mínimo, máximo y promedio general de las variables necesarias
    df_statistics = df_sim[['MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET', 'MinHmd', 'MaxHmd', 'Radsolar']]
    overall_min = df_statistics.min().to_frame().T.add_suffix('_min')
    overall_max = df_statistics.max().to_frame().T.add_suffix('_max')
    overall_mean = df_statistics.mean().to_frame().T.add_suffix('_mean')

    # Concatenar los resultados en un único dataframe
    overall_stats = pd.concat([overall_min, overall_max, overall_mean], axis=1)

    # Encuentra la fecha máxima y resta tres meses para obtener la fecha de inicio del último trimestre
    last_quarter_start = df_sim['Date'].max() - pd.DateOffset(months=3)

    # Filtrar datos del último trimestre
    last_quarter_data = df_sim[df_sim['Date'] >= last_quarter_start]

    # Calcular estadísticas generales del último trimestre
    df_last_quarter_data = last_quarter_data[['MinTemp', 'MaxTemp', 'Precipitation', 'ReferenceET',
                                              'MinHmd', 'MaxHmd', 'Radsolar']]
    last_quarter_min = df_last_quarter_data.min().to_frame().T.add_suffix('_last_quarter_min')
    last_quarter_max = df_last_quarter_data.max().to_frame().T.add_suffix('_last_quarter_max')
    last_quarter_mean = df_last_quarter_data.mean().to_frame().T.add_suffix('_last_quarter_mean')

    # Concatenar los resultados en un único dataframe
    last_quarter_stats = pd.concat([last_quarter_min, last_quarter_max, last_quarter_mean], axis=1)

    # Dataframe resultados aquacrop
    prod_potencial = pd.DataFrame({'estacion': filter_est,
                                   'start_date': df_sim['Date'].min().strftime('%Y/%m/%d'),
                                   'harvest_date': model.get_simulation_results()['Harvest Date (YYYY/MM/DD)'],
                                   'harvest_steps': model.get_simulation_results()['Harvest Date (Step)'],
                                   'dry_yield': model.get_simulation_results()['Dry yield (tonne/ha)'],
                                   'fresh_yield': model.get_simulation_results()['Fresh yield (tonne/ha)'],
                                   'yield_potencial': model._outputs.final_stats['Yield potential (tonne/ha)'],
                                   'seasonal_irrigation': model._outputs.final_stats['Seasonal irrigation (mm)'],
                                   'type_soil': type_soil})

    # Concatenar toda la información calculada
    prod_potencial = pd.concat([prod_potencial, overall_stats, last_quarter_stats], axis=1)

    # Dataframe final
    df_prod_final = pd.concat([df_prod_final, prod_potencial])
    df_prod_final = df_prod_final[~df_prod_final['fresh_yield'].isna()]

# Renombrar columnas en el DataFrame original
df_prod_final.rename(columns={**dict_target, **dict_features}, inplace=True)

# Renombrar el tipo de suelo a español
df_prod_final['Tipo_suelo'] = df_prod_final['Tipo_suelo'].map(dict_soil)

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [00:16<00:00,  3.05it/s]


In [ ]:
print(df_prod_final.shape)
df_prod_final.head()

(49, 51)


,estacion,start_date,harvest_date,harvest_steps,dry_yield,Rendimiento_estimado,yield_potencial,Riego_estacional,Tipo_suelo,TempMin_min,...,MinHumedRel_ult_trim_max,MaxHumedRel_ult_trim_max,RadSolar_ult_trim_max,TempMin_ult_trim_promedio,TempMax_ult_trim_promedio,Precipitacion_ult_trim_promedio,Evapotranspiracion_ult_trim_promedio,MinHumedRel_ult_trim_promedio,MaxHumedRel_ult_trim_promedio,RadSolar_ult_trim_promedio
0,MEL,2021/01/01,2021-12-31,363,31.900523,106.335077,37.192625,960,Franco Arcilloso,13.4,...,69.0,100.0,569.0,18.743011,30.908602,5.155914,4.504301,52.559140,100.000000,423.916129
0,PTA,2021/01/01,2021-12-31,363,1.965640,6.552133,37.192096,660,Arcilloso,15.5,...,67.0,100.0,635.8,19.624731,30.417204,4.701075,4.629032,51.548387,99.311828,436.587097
0,CAN,2021/01/01,2021-12-31,363,31.900029,106.333429,37.192096,1095,Franco Arcilloso,13.9,...,68.0,100.0,611.7,18.676344,31.039785,5.234409,4.831183,48.903226,99.849462,453.273118
0,TIP,2021/01/01,2021-12-31,363,31.899811,106.332703,37.191863,1005,Franco Arcilloso,14.1,...,67.0,100.0,617.0,18.961290,30.618280,4.977419,5.250538,51.612903,100.000000,465.704301
0,ORT,2021/01/01,2021-12-31,363,31.899921,106.333071,37.191981,915,Franco Arcilloso,14.9,...,70.0,100.0,605.2,18.759140,30.361290,5.812903,4.762366,52.150538,100.000000,454.208602


# Predicción AA

In [ ]:
# Codificación dummy de la columna categórica
dummies = pd.get_dummies(df_prod_final['Tipo_suelo'], prefix='Tipo_suelo', dtype='int', drop_first=False)

# Eliminar columna 'Tipo_suelo_Arcilloso' (categoría de referencia) para evitar redundancia, ya que una columna puede predecirse con las demás
dummies.drop('Tipo_suelo_Arcilloso', axis=1, inplace=True)

# Eliminar columna 'Tipo_suelo' en el dataframe original
df_pred = df_prod_final.drop('Tipo_suelo', axis=1)

# Agregar las columnas dummies nuevamente al DataFrame original
df_pred = pd.concat([df_pred, dummies], axis=1)

In [ ]:
# Nombre de las covariables
reject_features = ['Precipitacion_min', 'MaxHumedRel_max', 'Precipitacion_ult_trim_min', 'MaxHumedRel_ult_trim_max']
features_names = [col for col in list(df_pred.columns[7:]) if col not in reject_features]

In [ ]:
# Percentiles para acotar las covariables y evitar outliers
percentile_ranges = {
 'Riego_estacional': (0, 90),
 'TempMin_min': (0, 97.5),
 'TempMax_min': (5, 100),
 'Evapotranspiracion_min': (7.5, 95),
 'MinHumedRel_min': (5, 97.5),
 'MaxHumedRel_min': (10, 100),
 'RadSolar_min': (0, 99),
 'TempMin_max': (10, 90),
 'TempMax_max': (5, 95),
 'Precipitacion_max': (0, 95),
 'Evapotranspiracion_max': (2, 98),
 'MinHumedRel_max': (0, 100),
 'RadSolar_max': (5, 100),
 'TempMin_promedio': (5, 95),
 'TempMax_promedio': (2, 99),
 'Precipitacion_promedio': (0, 95),
 'Evapotranspiracion_promedio': (5, 97.5),
 'MinHumedRel_promedio': (2, 95),
 'MaxHumedRel_promedio': (15, 100),
 'RadSolar_promedio': (2, 100),
 'TempMin_ult_trim_min': (0, 100),
 'TempMax_ult_trim_min': (5, 95),
 'Evapotranspiracion_ult_trim_min': (5, 100),
 'MinHumedRel_ult_trim_min': (5, 95),
 'MaxHumedRel_ult_trim_min': (10, 100),
 'RadSolar_ult_trim_min': (0, 100),
 'TempMin_ult_trim_max': (2, 95),
 'TempMax_ult_trim_max': (5, 100),
 'Precipitacion_ult_trim_max': (0, 99),
 'Evapotranspiracion_ult_trim_max': (2, 98),#
 'MinHumedRel_ult_trim_max': (1, 95),
 'RadSolar_ult_trim_max': (1, 100),
 'TempMin_ult_trim_promedio': (7.5, 95),
 'TempMax_ult_trim_promedio': (5, 98),
 'Precipitacion_ult_trim_promedio': (0, 95),
 'Evapotranspiracion_ult_trim_promedio': (5, 95),
 'MinHumedRel_ult_trim_promedio': (5, 95),
 'MaxHumedRel_ult_trim_promedio': (15, 100),
 'RadSolar_ult_trim_promedio': (2, 100),
 'Tipo_suelo_Franco Arcilloso': (0, 100),
 'Tipo_suelo_Franco Arenoso': (0, 100),
 'Tipo_suelo_Arcillo Limoso': (0, 100),
 'Tipo_suelo_Franco Arcillo Limoso': (0, 100)
 }

In [ ]:
# Aplicar función para reemplazar outliers
df_pred = replace_outliers_with_percentiles(df_pred, percentile_ranges, features_names)

In [59]:
# Cargar el escalador para estandarizar las covariables
infile = open(models_path + "scaler_object.pkl", 'rb')
scaler = pickle.load(infile)

In [ ]:
# Estandarizar las covariables
df_pred_final = pd.DataFrame(scaler.transform(df_pred[df_pred.columns[7:]]),columns = df_pred.columns[7:])

In [ ]:
# Selección de variables finales
cols_cb = [
'Riego_estacional',
'Tipo_suelo_Franco Arcilloso',
'Tipo_suelo_Franco Arcillo Limoso',
'Tipo_suelo_Arcillo Limoso',
'Precipitacion_promedio',
'Precipitacion_ult_trim_promedio',
'TempMin_ult_trim_min',
'Evapotranspiracion_promedio',
'Tipo_suelo_Franco Arenoso'
]

# Filtrar dataframe con columnas finales
df_pred_final = df_pred_final[cols_cb]

# Predecir valores de regresión
ypred = cat_model_f.predict(df_pred_final)

# Redondear la predicción en una lista
rendimiento_aa = [round(y,2) for y in ypred]

In [ ]:
df_unique['Rendimiento_aquacrop'] = [round(y,2) for y in df_prod_final['Rendimiento_estimado']]
df_unique['Rendimiento_aa'] = rendimiento_aa

In [ ]:
df_unique

,Estacion,Hoja2__TEXTURA,Rendimiento_aquacrop,Rendimiento_aa
0,MELENDEZ,Franco Arcillosa,106.34,103.31
1,PTAR CALI,Arcillosa,6.55,9.25
2,CANDELARIA,Franco Arcillosa,106.33,103.88
3,EL TIPLE,Franco Arcillosa,106.33,101.04
4,ORTIGAL,Franco Arcillosa,106.33,102.38
5,MELENDEZ,Franco Arenosa,106.34,102.34
6,EL TIPLE,Arcillo Limosa,66.81,66.87
7,EL TIPLE,Franco Arenosa,106.33,96.84
8,EL TIPLE,Arcillosa,6.03,21.91
9,MELENDEZ,Arcillo Limosa,66.40,71.59


# Exportar raster con predicciones

In [ ]:
# Unir los valores predichos con el dataframe original en función de la estación y el suelo
df_raster_predictions = df_raster.merge(df_unique, on=['Estacion', 'Hoja2__TEXTURA'], how='left')

In [ ]:
df_raster_predictions

,OID_,pointid,grid_code,Hoja2__TEXTURA,Feature_poli1,Estacion,Rendimiento_aquacrop,Rendimiento_aa
0,1,14444,1,Franco Arcillosa,6,MELENDEZ,106.34,103.31
1,10,14453,3,Arcillosa,5,PTAR CALI,6.55,9.25
2,100,14543,1,Franco Arcillosa,7,CANDELARIA,106.33,103.88
3,1000,15960,1,Franco Arcillosa,6,MELENDEZ,106.34,103.31
4,10000,28735,1,Franco Arcillosa,4,EL TIPLE,106.33,101.04
...,...,...,...,...,...,...,...,...
42130,9995,28730,1,Franco Arcillosa,4,EL TIPLE,106.33,101.04
42131,9996,28731,1,Franco Arcillosa,4,EL TIPLE,106.33,101.04
42132,9997,28732,1,Franco Arcillosa,4,EL TIPLE,106.33,101.04
42133,9998,28733,1,Franco Arcillosa,4,EL TIPLE,106.33,101.04


In [ ]:
# Exportar dataframe a un archivo .txt con codificación UTF-8
df_raster_predictions.to_csv(output_path + 'Raster_predicciones.txt', sep='\t', index=False, encoding='utf-16')